In [1]:
import numpy as np
import sys
import matplotlib.pyplot as plt
sys.path.append('../SourceCode')
import bastien_utils
from shallowAE import ShallowAE
import datetime
import morphoMaths
from MaxPlusDecoder.ShallowAE_maxPlusDecoder.shallowAE_maxplus import ShallowAE_MaxPlus
from shallowAE import ShallowAE
from keras.models import Model, load_model
from keras.layers import Input, Dense, Flatten, Reshape, LeakyReLU, BatchNormalization, Activation, Dropout
from keras import losses, regularizers, metrics
import keras.utils 
import custom_regularizers

Using TensorFlow backend.


In [3]:
AE = ShallowAE_MaxPlus.load('18_12_04_dim100.h5', path_to_model_directory='../Results/MNIST_Fashion/ShallowAE_MaxPlus/')

In [4]:
AE.decoder.layers

In [7]:
AE.decoder.layers[1].get_config()['rate']

0.5

In [10]:
AE = ShallowAE()

In [6]:
latent_dim = 100
nb_input_channels=1
dropout_rate=None
nb_rows=28
nb_columns=28
input_img = Input(shape=(nb_rows, nb_columns, nb_input_channels))  # adapt this if using `channels_first` image data format
x = Flatten()(input_img)
encoded = Dense(latent_dim, activation='sigmoid', name='encoder_hidden_dense_layer',
              activity_regularizer=custom_regularizers.sparsity_Hoyer(beta=0.87, 
                                                                      sp_o=0.005))(x)
encoder = Model(input_img, encoded, name='encoder')
encoded_img = Input(shape=(latent_dim,))
if dropout_rate is None:
    x = Dense(nb_rows*nb_columns*1, use_bias=False)(encoded_img)
else:
    x = Dropout(dropout_rate, name='dropout_layer')(encoded_img)
    x = Dense(nb_rows*nb_columns*1, use_bias=False, name='name_test')(x)
decoded = Reshape((nb_rows,nb_columns,1))(x)
decoder = Model(encoded_img, decoded, name='decoder')  
encoded = encoder(input_img)
decoded = decoder(encoded)
AE = Model(input_img, decoded)
AE.compile(optimizer='adadelta', loss='mean_squared_error', metrics=['mse'])

In [3]:
encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
encoder_dense_layer (Dense)  (None, 100)               78500     
Total params: 78,500
Trainable params: 78,500
Non-trainable params: 0
_________________________________________________________________


In [8]:
encoder.get_layer(name='encoder_hidden_dense_layer').get_config()['activity_regularizer']['config']['beta']

0.8700000047683716

In [10]:
encoder.get_layer(name='encoder_hidden_dense_layer').get_config()['activity_regularizer']['class_name']

'sparsity_Hoyer'

In [33]:
decoder.get_layer(name='dropout_layer').get_config()['rate']

ValueError: No such layer: dropout_layer